<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 250px; display: inline" alt="IMT"/> </a>
</center>

In [1]:
#Importation des librairies utilisées
import unicodedata 
import time
import pandas as pd
import numpy as np
import random
import nltk
import collections
import itertools
import csv
import warnings
import scipy

from sklearn.cross_validation import train_test_split

/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Répertoire de travail
DATA_DIR = "/Users/bguillouet/Insa/TP_Insa/data/cdiscount_NLP/"
DATA_TFIDF_DIR = "/Users/bguillouet/Insa/TP_Insa/data/cdiscount_NLP/tfidf/"

# Nom des fichiers
training_reduit_path = DATA_DIR + "Categorie_reduit.csv"
# Variable Globale
HEADER_TEST = ['Description','Libelle','Marque']
HEADER_TRAIN =['Categorie1','Categorie2','Categorie3','Description','Libelle','Marque']

## 3 Construction des caractéristiques ou *features* (TF-IDF)¶
### Introduction
La vectorisation, c'est-à-dire la construction des caractéristiques à partir de la liste des mots se fait en 2 étapes:
* **Hashage**. Il permet de réduire l'espace des variables (taille du dictionnaire) en un nombre limité et fixé a priori `n_hash` de caractéristiques. Il repose sur la définition d'une fonction de hashage, $h$ qui à un indice $j$ défini dans l'espace des entiers naturels, renvoie un indice $i=h(j)$ dans dans l'espace réduit (1 à n_hash) des caractéristiques. Ainsi le poids de l'indice $i$, du nouvel espace, est l'association de tous les poids d'indice $j$ tels que $i=h(j)$ de l'espace originale. Ici, les poids sont associés d'après la méthode décrite par Weinberger et al. (2009).

N.B. $h$ n'est pas généré aléatoirement. Ainsi pour un même fichier d'apprentissage (ou de test) et pour un même entier n_hash, le résultat de la fonction de hashage est identique

* **TF-IDF**. Le TF-IDF permet de faire ressortir l'importance relative de chaque mot $m$ (ou couples de mots consécutifs) dans un texte-produit ou un descriptif $d$, par rapport à la liste entière des produits. La fonction $TF(m,d)$ compte le nombre d'occurences du mot $m$ dans le descriptif $d$. La fonction $IDF(m)$ mesure l'importance du terme dans l'ensemble des documents ou descriptifs en donnant plus de poids aux termes les moins fréquents car considérés comme les plus discriminants (motivation analogue à celle de la métrique du chi2 en anamlyse des correspondance). $IDF(m,l)=\log\frac{D}{f(m)}$ où $D$ est le nombre de documents, la taille de l'échantillon d'apprentissage, et $f(m)$ le nombre de documents ou descriptifs contenant le mot $m$. La nouvelle variable ou *features* est $V_m(l)=TF(m,l)\times IDF(m,l)$.

* Comme pour les transformations des variables quantitatives (centrage, réduction), la même transformation c'est-à-dire les mêmes pondérations, est calculée sur l'achantillon d'apprentissage et appliquée à celui de test.

### Fonction de Vectorisation

In [3]:
## Création d’une matrice indiquant
## les fréquences des mots contenus dans chaque description
## de nombreux paramètres seraient à tester
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction import FeatureHasher


def vectorizer_train(df, columns=['Description', 'Libelle', 'Marque'], nb_hash=None, stop_words=None):
    
    # Hashage
    if nb_hash is None:
        data_hash = map(lambda x : " ".join(x), df[columns].values)
        feathash = None
            # TFIDF
        vec = TfidfVectorizer(
            min_df = 1,
            stop_words = stop_words,
            smooth_idf=True,
            norm='l2',
            sublinear_tf=True,
            use_idf=True,
            ngram_range=(1,1)) #bi-grams
        tfidf = vec.fit_transform(data_hash)
    else:
        df_text = map(lambda x : collections.Counter(" ".join(x).split(" ")), df[columns].values)
        feathash = FeatureHasher(nb_hash)
        data_hash = feathash.fit_transform(map(collections.Counter,df_text))
        
        vec =  TfidfTransformer(use_idf=True,
                            smooth_idf=True, sublinear_tf=False)
        tfidf =  vec.fit_transform(data_hash)

    return vec, feathash, tfidf



def apply_vectorizer(df, vec, columns =['Description', 'Libelle', 'Marque'], feathash = None ):
    
    #Hashage
    if feathash is None:
        data_hash = map(lambda x : " ".join(x), df[columns].values)
    else:
        df_text = map(lambda x : collections.Counter(" ".join(x).split(" ")), df[columns].values)
        data_hash = feathash.transform(df_text)
    
    # TFIDF
    tfidf=vec.transform(data_hash)
    return tfidf


In [4]:
data_valid_clean = pd.read_csv(DATA_DIR + "Categorie_reduit_valid_clean.csv").fillna("")
data_train_clean = pd.read_csv(DATA_DIR + "Categorie_reduit_train_clean.csv").fillna("")

#for nb_hash in [30]:
for nb_hash in [None, 300, 1000, 10000, 100000]:
    print(nb_hash)
    vec, feathash, X = vectorizer_train(data_train_clean, columns=["Description"], nb_hash = nb_hash)
    Xv = apply_vectorizer(data_valid_clean, vec, feathash=feathash)
    scipy.sparse.save_npz(DATA_TFIDF_DIR + "Embedded_idf_Categorie_reduit_train_clean_N_hash_"+str(nb_hash), X)
    scipy.sparse.save_npz(DATA_TFIDF_DIR + "Embedded_idf_Categorie_reduit_valid_clean_N_hash_"+str(nb_hash), Xv)


None
300
1000
10000
100000


In [5]:
data_valid_clean = pd.read_csv(DATA_DIR + "Categorie_reduit_valid_clean_stem.csv").fillna("")
data_train_clean = pd.read_csv(DATA_DIR + "Categorie_reduit_train_clean_stem.csv").fillna("")

#for nb_hash in [30]:
for nb_hash in [None, 300, 1000, 10000, 100000]:
    print(nb_hash)
    vec, feathash, X = vectorizer_train(data_train_clean, columns=["Description"], nb_hash = nb_hash)
    Xv = apply_vectorizer(data_valid_clean, vec, feathash=feathash)
    scipy.sparse.save_npz(DATA_TFIDF_DIR + "Embedded_idf_Categorie_reduit_train_clean_N_hash_stem_"+str(nb_hash), X)
    scipy.sparse.save_npz(DATA_TFIDF_DIR + "Embedded_idf_Categorie_reduit_valid_clean_N_hash_stem_"+str(nb_hash), Xv)

None
300
1000
10000
100000
